In [1]:
!pip3 install pandas opencv-python numpy matplotlib scikit-learn Pillow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# %load_ext cudf.pandas
import pandas as pd
# import cudf as pd
from matplotlib import pyplot as plt

/home/menaf/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
dataset_path = "./raw_data/"

train_cats_df = pd.read_csv(dataset_path + "train/categories.csv")
train_df = pd.read_csv(dataset_path + "train/train_labels.csv")

test_cats_df = pd.read_csv(dataset_path + "test/categories.csv")
# test_df = pd.read_csv(dataset_path + "test/test_labels.csv")

In [5]:
print(train_df)

                 image_name  \
0                    00.png   
1                    01.png   
2                    02.png   
3                    03.png   
4                    04.png   
...                     ...   
2143  3dom_fbk_img_1554.png   
2144           img_0257.png   
2145  3dom_fbk_img_1538.png   
2146           img_0182.png   
2147           img_0055.png   

                                        rotation_matrix  \
0     0.999017467386748;-0.01951432487219089;0.03979...   
1     0.999147719991382;-0.021624129414769648;0.0351...   
2     0.9992527616183833;-0.02402019259931326;0.0302...   
3     0.9993946226667176;-0.02356062921667625;0.0255...   
4     0.9995276708105233;-0.02256816267742356;0.0208...   
...                                                 ...   
2143  0.01591948448470304;-0.9942358284779308;0.1060...   
2144  0.7798812386299148;-0.11966528279975605;-0.614...   
2145  0.212343457025313;-0.9771913266348059;-0.00271...   
2146  0.9949298086968184;-0.025133801

In [6]:
grouped_train_df = train_df.groupby("dataset")
print(grouped_train_df)

In [8]:
from itertools import product
# creating a new dataset with pairwise images and relative extrinsics
# a pair with image A, B, extrinsics K_a, K_b will follow this convention

# A -> B (image 1: A, image 2: B), K_rel = K_b - K_a (rel extrinsics to get you from A to B)
# B -> A vice versa
group_rows = {}

def construct_pair(r1, r2):
    attrs = (r1.index.to_list())
    combined_row = {}
    for attr in attrs:
        if attr not in ["dataset", "scene"]:
            combined_row[attr + "_1"] = r1[attr]
            combined_row[attr + "_2"] = r2[attr]
        else:
            combined_row[attr] = r1[attr]
            
    return combined_row

all_pairs = []
for group_name, group in grouped_train_df:
    print(f"Grouping: {group_name}")
    
    group_rows = [group.iloc[i] for i in range(len(group))]
    pairs = list(product(group_rows, repeat=2))
    for r1, r2 in pairs:
        # print(r1, r2)
        
        pair = construct_pair(r1, r2)
        all_pairs.append(pair)
    print(len(pairs))
    print(len(group))
    
    
print("Total Pairs: ", len(all_pairs))

Grouping: church
12100
110
Grouping: dioscuri
4900
70
Grouping: lizard
505521
711
Grouping: multi-temporal-temple-baalshamin
4624
68
Grouping: pond
1247689
1117
Grouping: transp_obj_glass_cup
1296
36
Grouping: transp_obj_glass_cylinder
1296
36
Total Pairs:  1777426


In [9]:
final_df = pd.DataFrame(all_pairs)

In [10]:
import numpy as np
# calculating relative camera extrinsics

def get_relative_extrinsics(R1, R2, T1, T2):
	mat1 = np.hstack((R1, T1))
	mat1 = np.concatenate((mat1, np.array([[0, 0, 0, 1]])))

	mat2 = np.hstack((R2.T, -R2.T @ T2))
	mat2 = np.concatenate((mat2, np.array([[0, 0, 0, 1]])))

	final = mat1 @ mat2
	return final
	# R_rel = final[:3, :3]
	# T_rel = final[:3, 3]
	# return 

print(final_df.columns)
def get_relative_extrinsics_from_row(row):
	R1 = np.array(row["rotation_matrix_1"].split(";"), dtype=np.float32).reshape(3, 3)
	R2 = np.array(row["rotation_matrix_2"].split(";"), dtype=np.float32).reshape(3, 3)
	
	T1 = np.array(row["translation_vector_1"].split(";"), dtype=np.float32).reshape(3,1)
	T2 = np.array(row["translation_vector_2"].split(";"), dtype=np.float32).reshape(3,1)
	return get_relative_extrinsics(R1, R2, T1, T2)


Index(['image_name_1', 'image_name_2', 'rotation_matrix_1',
       'rotation_matrix_2', 'translation_vector_1', 'translation_vector_2',
       'calibration_matrix_1', 'calibration_matrix_2', 'dataset', 'scene'],
      dtype='object')


In [11]:
# !pip install \
#     --extra-index-url=https://pypi.nvidia.com \
#     cudf-cu11==24.4.* dask-cudf-cu11==24.4.* cuml-cu11==24.4.* \
#     cugraph-cu11==24.4.* cuspatial-cu11==24.4.* cuproj-cu11==24.4.* \
#     cuxfilter-cu11==24.4.* cucim-cu11==24.4.* pylibraft-cu11==24.4.* \
#     raft-dask-cu11==24.4.* cuvs-cu11==24.4.*

In [12]:
# import cudf

# final_cudf = cudf.DataFrame.from_pandas(final_df)
final_df['relative_extrinsics'] = final_df.apply(get_relative_extrinsics_from_row, axis=1)
print(final_df["relative_extrinsics"])

0          [[1.0000000162967462, -3.074028143146279e-09, ...
1          [[-0.08756732734125183, 0.11335737261337897, 0...
2          [[0.5954032970502479, 0.4028578703572556, 0.69...
3          [[0.9919287498048313, 0.006586841193185311, -0...
4          [[0.36703325231942685, 0.49121953173173516, 0....
                                 ...                        
1777421    [[0.9980171979564142, 0.06114615018315134, -0....
1777422    [[0.9988630238350997, 0.045077481379170725, -0...
1777423    [[0.9994945817143017, 0.029038827083638585, -0...
1777424    [[0.999878518314532, 0.013755256597761667, -0....
1777425    [[0.9999999495562844, 6.18392004270163e-11, -4...
Name: relative_extrinsics, Length: 1777426, dtype: object


In [14]:
# save the newly created dataset into a directly called paired_dataset
final_df.to_csv("./preprocessed_data/paired_dataset.csv", index=False)